___

<a href='http://www.pieriandata.com'> <img src='../Pierian_Data_Logo.png' /></a>
___

# Non-Negative Matric Factorization

Let's repeat thet opic modeling task from the previous lecture, but this time, we will use NMF instead of LDA.

## Data

We will be using articles scraped from NPR (National Public Radio), obtained from their website [www.npr.org](http://www.npr.org)

In [1]:
import pandas as pd

In [2]:
npr = pd.read_csv('bible.csv')

In [3]:
npr.head()

,Book,Content
0,Genesis,"In the beginning, God created the heavens and ..."
1,Exodus,These are the names of the sons of Israel who ...
2,Leviticus,The LORD called Moses and spoke to him from th...
3,Numbers,The LORD spoke to Moses in the wilderness of S...
4,Deuteronomy,These are the words that Moses spoke to all Is...


Notice how we don't have the topic of the articles! Let's use LDA to attempt to figure out clusters of the articles.

## Preprocessing

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [5]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [6]:
dtm = tfidf.fit_transform(npr['Content'])

In [7]:
dtm

<66x7619 sparse matrix of type '<class 'numpy.float64'>'
	with 58977 stored elements in Compressed Sparse Row format>

## NMF

In [8]:
from sklearn.decomposition import NMF

In [9]:
nmf_model = NMF(n_components=7)

In [10]:
# This can take awhile, we're dealing with a large amount of documents!
nmf_model.fit(dtm)

NMF(n_components=7)

## Displaying Topics

In [11]:
len(tfidf.get_feature_names())

7619

In [12]:
import random

In [13]:
for i in range(10):
    random_word_id = random.randint(0,7619)
    print(tfidf.get_feature_names()[random_word_id])

loudly
nullify
heap
flashing
country
enter
dishes
ebed
pastured
defrauded


In [14]:
len(nmf_model.components_)

7

In [15]:
nmf_model.components_

array([[4.44539350e-04, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.63039524e-02, 4.16133406e-03, 4.16133406e-03, ...,
        8.08404042e-03, 1.79731661e-03, 6.06733188e-03],
       ...,
       [2.53670668e-02, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.60964237e-04, 0.00000000e+00, 2.63181746e-04],
       [0.00000000e+00, 1.75433108e-04, 1.75433108e-04, ...,
        0.00000000e+00, 4.10316675e-05, 2.85626434e-04]])

In [16]:
len(nmf_model.components_[0])

7619

In [17]:
single_topic = nmf_model.components_[0]

In [18]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([3809, 4218, 4217, ..., 4028, 3385, 4099], dtype=int64)

In [19]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([3704, 7607, 1748, 4524, 5865, 1704, 6024, 4028, 3385, 4099],
      dtype=int64)

In [20]:
top_word_indices = single_topic.argsort()[-10:]

In [21]:
for index in top_word_indices:
    print(tfidf.get_feature_names()[index])

jerusalem
zion
declares
nations
says
day
shall
like
hosts
lord


These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [36]:
for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-5:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['day', 'shall', 'like', 'hosts', 'lord']


THE TOP 15 WORDS FOR TOPIC #1
['grace', 'lord', 'faith', 'jesus', 'christ']


THE TOP 15 WORDS FOR TOPIC #2
['david', 'israel', 'son', 'sons', 'king']


THE TOP 15 WORDS FOR TOPIC #3
['went', 'came', 'disciples', 'jesus', 'said']


THE TOP 15 WORDS FOR TOPIC #4
['wise', 'wicked', 'wisdom', 'like', 'man']


THE TOP 15 WORDS FOR TOPIC #5
['people', 'israel', 'land', 'lord', 'shall']


THE TOP 15 WORDS FOR TOPIC #6
['beloved', 'does', 'love', 'abides', 'truth']




### Attaching Discovered Topic Labels to Original Articles

In [37]:
dtm

<66x7619 sparse matrix of type '<class 'numpy.float64'>'
	with 58977 stored elements in Compressed Sparse Row format>

In [38]:
dtm.shape

(66, 7619)

In [39]:
len(npr)

66

In [41]:
topic_results = nmf_model.transform(dtm)

In [42]:
topic_results.shape

(66, 7)

In [43]:
topic_results[0]

array([0.        , 0.        , 0.21299579, 0.25855183, 0.04834474,
       0.15068378, 0.        ])

In [44]:
topic_results[0].round(2)

array([0.  , 0.  , 0.21, 0.26, 0.05, 0.15, 0.  ])

In [45]:
topic_results[0].argmax()

3

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [49]:
print(npr)

           Book                                            Content  Topic
0       Genesis  In the beginning, God created the heavens and ...      3
1        Exodus  These are the names of the sons of Israel who ...      5
2     Leviticus  The LORD called Moses and spoke to him from th...      5
3       Numbers  The LORD spoke to Moses in the wilderness of S...      5
4   Deuteronomy  These are the words that Moses spoke to all Is...      5
..          ...                                                ...    ...
61       1 John  That which was from the beginning, which we ha...      6
62       2 John  The elder to the elect lady and her children, ...      6
63       3 John  The elder to the beloved Gaius, whom I love in...      6
64         Jude  Jude, a servant of Jesus Christ and brother of...      1
65   Revelation  The revelation of Jesus Christ, which God gave...      4

[66 rows x 3 columns]


In [32]:
topic_results.argmax(axis=1)

array([3, 5, 5, 5, 5, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 0, 4, 4, 4,
       5, 0, 0, 5, 5, 5, 0, 5, 5, 0, 5, 0, 0, 5, 0, 0, 0, 3, 3, 3, 3, 3,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 6, 6, 6, 1, 4],
      dtype=int64)

In [33]:
npr['Topic'] = topic_results.argmax(axis=1)

In [51]:
npr.head(60)

,Book,Content,Topic
0,Genesis,"In the beginning, God created the heavens and ...",3
1,Exodus,These are the names of the sons of Israel who ...,5
2,Leviticus,The LORD called Moses and spoke to him from th...,5
3,Numbers,The LORD spoke to Moses in the wilderness of S...,5
4,Deuteronomy,These are the words that Moses spoke to all Is...,5
5,Joshua,After the death of Moses the servant of the LO...,2
6,Judges,"After the death of Joshua, the people of Israe...",2
7,Ruth,In the days when the judges ruled there was a ...,3
8,1 Samuel,There was a certain man of Ramathaim-zophim of...,2
9,2 Samuel,"After the death of Saul, when David had return...",2


## Great work!